In [10]:
import os
import ee
import numpy as np
from geeml.extract import extractor
import pandas as pd
# import geemap
# Authenticate GEE
# ee.Authenticate()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"

service_account = 'gee-serdp-upload@appspot.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")
ee.Initialize(credentials)
# Initialize GEE with high-volume end-point
# ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Initialize()


In [11]:
import geemap
import os
from google.cloud import storage
from google.cloud import client


In [12]:
os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json"
storage_client = storage.Client.from_service_account_json("/explore/nobackup/people/spotter5/cnn_mapping/gee-serdp-upload-7cd81da3dc69.json")

os.environ["GCLOUD_PROJECT"] = "gee-serdp-upload"
storage_client = storage.Client()
# bucket_name = 'smp-scratch/mtbs_1985'
bucket_name = 'smp-scratch'

bucket = storage_client.bucket(bucket_name)

Import assetts of interest

In [13]:
geometry = ee.FeatureCollection('users/spotter/fire_cnn/raw/eurasia') #area of interest
mod1 = ee.ImageCollection("MODIS/061/MOD10A1") #active fire
mod2 = ee.ImageCollection("MODIS/061/MYD10A1") #active fire
fire_cci = ee.ImageCollection("ESA/CCI/FireCCI/5_1") #active fire
mod_burn = ee.ImageCollection("MODIS/061/MCD64A1") #mcd64a1
sent_2A = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") #sentinel 2
s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY') #cloud masking for sentinel
lfdb = ee.FeatureCollection("users/spotter/fire_cnn/raw/nbac_1985") #nbac_fire_polygons, this can be any polygon shapefile, final version would be nbac and mtbs
# input_grid = ee.FeatureCollection('users/spotter/fire_cnn/raw/eurasia_dnbr_grid_clip').union() #grid to loop through to get around memory errors


In [16]:
terra = ee.ImageCollection('MODIS/006/MOD09GA')
aqua = ee.ImageCollection('MODIS/006/MYD09GA')


year = 2015


# //modis masking
# var getQABits = function(image, start, end, newName) {
#     // Compute the bits we need to extract.
#     var pattern = 0;
#     for (var i = start; i <= end; i++) {
#       pattern += Math.pow(2, i);
#     }
#     return image.select([0], [newName])
#                   .bitwiseAnd(pattern)
#                   .rightShift(start);
# };


#modis masking
# def getQABits(image, start, end, newName):
#     pattern = 0
    
#     for i in range(end):
        
#         # pattern = Math.pow(2, i)
#         pattern = 2 ** i

        
#         image = image.select([0], [newName]).bitwiseAnd(pattern).rightShift(start)
        
#         return image

def getQABits(image, start, end, new_name):
    # Compute the bits we need to extract.
    pattern = sum([2**i for i in range(start, end + 1)])
    
    return image.select([0], [new_name]) \
                .bitwise_and(pattern) \
                .right_shift(start)
    

# //combine terra and aqua
# var modis_sr = modis_sr.map(function(image){
    
#       // Select the QA band
#       var QA = image.select('QC_500m');
      
#       // Get the quality bits.
#       var qual = getQABits(QA, 0, 1, 'qual_flag')
#                           .expression("b(0) == 1 || b(0) == 2");
      
#       var QA1 = image.select('state_1km');
      
#       // Get the cloud_state bits and find cloudy areas.
#       var cloud = getQABits(QA1, 0, 1, 'cloud_state')
#                           .expression("b(0) == 1 || b(0) == 2");
      
#       // Get the land_water_flag bits.
#       var landWaterFlag = getQABits(QA1, 3, 5, 'land_water_flag');
      
#       // Get the snow_flag bits.
#       var snowFlag = getQABits(QA1, 15, 15, 'snow_flag');
      
#       // Create a mask that filters out deep ocean and cloudy areas.
#       var mask = landWaterFlag.neq(7).and(cloud.not()).and(qual.eq(0).and(snowFlag.eq(0)));
      
#       return image.updateMask(mask)//.select(['sur_refl_b01','sur_refl_b02','sur_refl_b05','sur_refl_b07']).clamp(-32768,32767)
#         //.addBands(image.select('num_observations_500m').clamp(-128,127)).copyProperties(image).set('system:time_start',image.get('system:time_start')).set('system:time_end',image.get('system:time_end'));
    
# });    

def modis_mask(image):
    
    #Select the QA band
    QA = image.select('QC_500m')
      
    #Get the quality bits.
    qual = getQABits(QA, 0, 1, 'qual_flag').expression("b(0) == 1 || b(0) == 2")
      
    QA1 = image.select('state_1km')
      
    #Get the cloud_state bits and find cloudy areas.
    cloud = getQABits(QA1, 0, 1, 'cloud_state').expression("b(0) == 1 || b(0) == 2")
      
    #Get the land_water_flag bits.
    landWaterFlag = getQABits(QA1, 3, 5, 'land_water_flag')
      
    #Get the snow_flag bits.
    snowFlag = getQABits(QA1, 15, 15, 'snow_flag')
      
    #Create a mask that filters out deep ocean and cloudy areas.
    mask = landWaterFlag.neq(7).And(cloud.Not()).And(qual.eq(0).And(snowFlag.eq(0)))
      
    return image.updateMask(mask)#.select(['sur_refl_b01','sur_refl_b02','sur_refl_b05','sur_refl_b07']).clamp(-32768,32767)

    
modis_sr = terra.merge(aqua).filterBounds(geometry)




In [20]:

years = [2015]
for year in years:
    
        
    
        # territory22 = input_grid.filter(ee.Filter.eq('Id',  territory))
        
    #create output image collection
    # base = 'projects/gee-serdp-upload/assets/cnn_mapping/nwt_nbac_2d_test2'
    base = 'users/spotter/fire_cnn/Russia/modis_dnbr'

    !earthengine create collection $base

    #full pathway to images within the collection
    full_out =os.path.join(base, str(year) + '_0901_1001')
    
    # !earthengine create collection $full_out


    #check if the asset exists, if it does skip it
    try:
        exists = ee.data.getAsset(full_out)

        print(f"Image {full_out} exists")

    except:
        print(f"Image {full_out} does not exist")


        pre_start_mod = str(year - 1) + '-09-01'
        pre_end_mod = str(year - 1) + '-10-01'

        #get post dates
        post_start_mod = str(year + 0) + '-09-01'
        post_end_mod = str(year + 0) + '-10-01'

        modis_sr_pre = modis_sr.filterDate(pre_start_mod, pre_end_mod).map(modis_mask).median().normalizedDifference(['sur_refl_b01','sur_refl_b07']).select([0], ['NBR']).cast({'NBR': 'float'})
        modis_sr_post = modis_sr.filterDate(post_start_mod, post_end_mod).map(modis_mask).median().normalizedDifference(['sur_refl_b01','sur_refl_b07']).select([0], ['NBR']).cast({'NBR': 'float'})

        mod_dnbr = modis_sr_pre.subtract(modis_sr_post).clip(geometry).multiply(1000).toShort()




        print(f"Downloading {year}")

        task = ee.batch.Export.image.toAsset(
                          # image = for_predict.multiply(1000).toShort(),
                          image = mod_dnbr,
                          region=geometry.geometry(), 
                          description= str(year) ,
                          scale=463,
                          crs='SR-ORG:6974',
                          # crs= 'EPSG:4326',
                          maxPixels=1e13,
                          assetId =  full_out)
                          # assetId =  'users/spotter/fire_cnn/mtbs_predictions/' + str(year) + '_' + str(territory22))


        task.start()



                        # except:
                        #     pass

Asset users/spotter/fire_cnn/Russia/modis_dnbr already exists.
Image users/spotter/fire_cnn/Russia/modis_dnbr/2015_0901_1001 does not exist


In [ ]:
't'